<span style="font-size:larger;">Introduction</span>
<tab></tab>
Hello there! This is my very first attempt at writing a kernel. And I hope that it is up to expectation.
I always welcome criticism, so please let me know what I can do to improve my skills.

After taking the fourth class of deeplearning.ai - Convolutional Neural Network, I wanted test it out on some other dataset. The reason I picked this dataset is because I have written a logistic regression using TensorFlow on the original and popular MNIST dataset, and I thought this dataset is pretty interesting. 

My approach to this problem is to first perform a simple CNN(just because I am excited for it) and then perform a logistic regression and compare them both. The reason behind picking logistic regression as a comparison is that in the previous MNIST problem(original), logistic regression could already perform pretty accurately, I managed to achieve 98% accuracy with just logistic regression. So if CNN is more powerful, it should be able to yield a much better accuracy.

Before diving into all that, let's take a look at the data and its distribution!


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
# Import data
data = pd.read_csv("../input/handwritten_data_785.csv", encoding = 'utf8')

The DataFrame as 378,037 Rows, and 785 Columns.

The first columns is the target values. Since we are predicting alphabets, it has 26 values ranging from 0 to 25. Each of the number relates to its corresponding alphabets. For example, 0 would be A, 1 would be B, and so on. 

Each row corresponds to an image, since the image pixel is 28 x 28, we have 28*28 = 784 numbers.

In [ ]:
print("DataFrame shape:" + str(data.shape))

The code below is just to make life simple and set a variable to X and Y.

target : target variables

featues: features variables

In [ ]:
target = data.iloc[:,0].values.reshape(-1,1)
features = data.iloc[:, 1:]
print("Target shape:" + str(target.shape))
print("Features shape:" + str(features.shape))

In [ ]:
pd.DataFrame(target)[0].value_counts(normalize=True).sort_values(ascending=False).plot(kind='bar')

The graph above is the distribution of the target alphabets. As we can see, the alphabvet 'O'  has the highet precentage with almost 16% where as 'V' has the least percentage. 

In [ ]:
# Import pacakges
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

We have to do a little data preprocessing.

First we have to set an input_shape variable, so that it will be easier to feed it into our model later.

We also have to convert our target variables into categorical variables and split the data into trainng and test set. 

I understand that Professor Andrew Ng said that we should have a training set, validation set and test set. But just for simplicity, and to do a quick dirty prototyping, I shall ignore the validation set for now. 

Next, we convert our image data from a row to a mx28x28x1 tensor and normalize it by performing an element-wise division by 255. 

In [ ]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

target2 = to_categorical(target.copy())

X_train, X_test, y_train, y_test = train_test_split(features, target2, test_size=0.3, random_state=42)

X_train = X_train.values.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.values.reshape(X_test.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255


Time to create a model!

In this simple model, first, I added a (3x3) padding to the original image and then convolve it using a (3 x 3) window with 32 filtes and (1x1) stride. Next, we perform batch normalization on the output and then feed it into a ReLU activation function. After that, we will perform a maxpooling to downsample the image. Finally, we flatten the data and feed it into a fully connected layer with a Softmax activation function. 

In [ ]:
def model(input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3,3))(X_input)
    X = Conv2D(32, (3,3), strides=(1,1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2), name='maxpool1')(X)
    X = Flatten()(X)
    X = Dense(26, activation='softmax', name = 'fc')(X)
    model = Model(inputs = X_input, outputs=X, name = 'ConvModel')
    return model

In [ ]:
firstModel = model(X_train.shape[1:])

In [ ]:
firstModel.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
firstModel.fit(X_train, y_train, epochs = 5, batch_size = 256)

In [ ]:
firstModel.evaluate(X_test, y_test)

In [ ]:
firstModel.summary()

As shown above, since the data is pretty huge, it takes approximately 10 minutes to finish running an epoch. The good thing is that the loss reduces steadily after each epoch and the accuracy rate increases. Seems like a pretty good model. 
You can glance through the model summary above. 

For this simple model, we manage to produce a 96.91% accuracy rate on the test set. Perhaps running on a longer epoch will increase the accuracy. 

The model below will explore a simple neural network to perform a 3 hidden layers softmax regression . Before doing that, let's split the train test set again and not reshaping them into a 3 dimensional array. 

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
inputs = X_train.shape[1]

In [ ]:
def logit_model(inputs):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim=inputs))
    model.add(Dense(256, activation='relu', input_dim=512))
    model.add(Dense(128, activation='relu', input_dim=256))
    model.add(Dense(26, activation='softmax', input_dim=128))
    return model

In [ ]:
log_reg = logit_model(inputs)

In [ ]:
log_reg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model = log_reg.fit(X_train, y_train, epochs = 5, batch_size = 128)

In [ ]:
test_score = log_reg.evaluate(X_test, y_test)

In [ ]:
print('Test cost: ' + str(test_score[0]))
print('Test cost: ' + str(test_score[1]))

This resulting is interesting. A 3 hidden layers softmax regression performs better than a CNN network.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(range(len(model.history['loss'])), model.history['loss'], label='Training cost')
plt.title('Training Cost')


In [ ]:
plt.plot(range(len(model.history['acc'])), model.history['acc'], 'r', label='Training Accuracy')
plt.title('Training Accuracy')